#**Entity Resolution**

In [ ]:
#importing necessary libraries

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [ ]:
!pip3 install recordlinkage #installing recordlinkage pakage

In [ ]:
#importing recordlinkagelibraries
from recordlinkage import Index
from recordlinkage.compare import String
import recordlinkage
from recordlinkage.preprocessing import phonetic

In [ ]:
# loading data from drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Reading Credit Bureau Data
originalDataT = pd.read_excel('/content/drive/MyDrive/entityRes/Customer Demographics - Credit Bureau.xlsx')

In [ ]:
#Reading Training/Testing Data
TrainData = pd.read_excel('/content/drive/MyDrive/entityRes/Training/Training Data.xlsx')
TrainGT = pd.read_excel('/content/drive/MyDrive/entityRes/Training/Training Data - Ground Truth.xlsx',)
TestData = pd.read_excel('/content/drive/MyDrive/entityRes/Test/Test Data.xlsx')
TestGT = pd.read_excel('/content/drive/MyDrive/entityRes/Test/Test Data - Ground Truth.xlsx')
Val = pd.read_excel('/content/drive/MyDrive/entityRes/Validation/Validation Data.xlsx')

In [ ]:
from datetime import datetime

def detect_date_format(date_str):
    formats_to_try = [
        "%Y-%m-%d %H:%M:%S",
        "%Y-%m-%d",
        "%m/%d/%Y",
        "%d/%m/%Y",
        "%m-%d-%Y",
        "%d-%m-%Y",
    ]

    for date_format in formats_to_try:
        try:
            parsed_date = datetime.strptime(date_str, date_format)
            return date_format
        except ValueError:
            pass

    return None  # If the format couldn't be detected


def convert_date_format(input_date):
  try:

 # Parse the input date string into a datetime object
    format = detect_date_format(input_date)
    if(format):
      datetime_obj = datetime.strptime(input_date,format)# '%Y-%m-%d %H:%M:%S')
    else:
      datetime_obj = datetime.strptime(input_date,'%Y-%m-%d %H:%M:%S')

    # Format the datetime object into the desired format
    formatted_date = datetime_obj.strftime('%Y-%m-%d %H:%M:%S')

    return formatted_date
  except ValueError:
        # Handle cases where the input does not match the expected format
        return input_date

In [ ]:
# Selecting Necesaary columns
TrainData['DOB'] = TrainData['DOB'].apply(lambda x: convert_date_format(str(x)))
columns_list = TrainData.columns.tolist()

Train = TrainData[['Cust ID', 'First Name', 'Last Name','Address Line 1','City', 'State', 'Zip', 'DOB']]
Test = TestData[['Cust ID', 'First Name', 'Last Name','Address Line 1','City', 'State', 'Zip', 'DOB']]
Val = Val[['Cust ID', 'First Name', 'Last Name','Address Line 1','City', 'State', 'Zip', 'DOB']]

originalData = originalDataT[['id', 'first_name', 'last_name','addr 1', 'addr 2', 'city', 'state', 'zip', 'DOB']]

In [ ]:
print(TrainData['DOB'] )# My fuction is not working. I am at my wits end to find the issue

0                           nan
1      Friday, Febraury 5, 1965
2           1983-03-15 00:00:00
3           2023-01-01 00:00:00
4           1981-01-01 00:00:00
                 ...           
155         1957-08-15 00:00:00
156         1948-12-31 00:00:00
157         1967-01-12 00:00:00
158         1999-01-12 00:00:00
159         1992-01-12 00:00:00
Name: DOB, Length: 160, dtype: object


In [ ]:
# matching with input data to train the model
train = TrainData.merge(TrainGT,on ='Cust ID')#left_on='Cust ID',right_on = 'Cust ID')
train.head()
train = train.merge(originalDataT,left_on='Credit Bureau ID',right_on='id')
train.head()
print(train.columns.tolist())

['Cust ID', 'First Name', 'Middle Name', 'Last Name', 'Prefix', 'Suffix', 'Address Line 1', 'Address Line 2', 'City', 'State', 'Zip', 'DOB_x', 'Credit Bureau ID', 'id', 'first_name', 'Middle name', 'last_name', 'prefix', 'suffix', 'addr 1', 'addr 2', 'city', 'state', 'zip', 'DOB_y']


In [ ]:
def jaccard_similarity(str1, str2):
    set1 = set(str1)
    set2 = set(str2)
    intersection = len(set1.intersection(set2))
    union = len(set1) + len(set2) - intersection
    similarity = intersection / union
    return similarity


In [ ]:
#Calculation Similarity

jaccard_similarity_val1 = train.apply(lambda row: jaccard_similarity(row['First Name'], row['first_name']), axis=1)
jaccard_similarity_val2 = train.apply(lambda row: jaccard_similarity(row['Last Name'], row['last_name']), axis=1)
#jaccard_similarity_val4 = train.apply(lambda row: jaccard_similarity(row['City'], row['city']), axis=1)
#jaccard_similarity_val5 = train.apply(lambda row: jaccard_similarity(row['State'], row['state']), axis=1)

jaccard_similarity_first =  jaccard_similarity_val1.sum()/len(jaccard_similarity_val1)
jaccard_similarity_last =  jaccard_similarity_val2.sum()/len(jaccard_similarity_val2)

In [ ]:
# changing index to match on index
Train.set_index('Cust ID', inplace=True)
originalData.set_index('id', inplace=True)

# Remove the 'ID' column from both DataFrames
#Train.drop(columns=['Cust ID'], inplace=True)
#originalData.drop(columns=['id'], inplace=True)



In [ ]:

from recordlinkage.index import Block

# Add a phonetic version of the first and last names to the two datasets
Train["phonetic_given_name"] = phonetic(Train["First Name"], "soundex")
originalData["phonetic_given_name"] = phonetic(originalData["first_name"], "soundex")

Train["phonetic_last_name"] = phonetic(Train["Last Name"], "soundex")
originalData["phonetic_last_name"] = phonetic(originalData["last_name"], "soundex")

Train["phonetic_city"] = phonetic(Train["City"], "soundex")
originalData["phonetic_city"] = phonetic(originalData["city"], "soundex")

Train["phonetic_state"] = phonetic(Train["State"], "soundex")
originalData["phonetic_state"] = phonetic(originalData["state"], "soundex")

#Train["initials"] = (Train["First Name"].str[0] + Train["Last Name"].str[0])
#originalData["initials"] = (originalData["first_name"].str[0] + originalData["last_name"].str[0])


indexer = recordlinkage.Index()
#indexer.block("initials")
indexer.add(Block('phonetic_given_name', 'phonetic_given_name'))
indexer.add(Block('phonetic_last_name', 'phonetic_last_name'))
candidate_links = indexer.index(Train,originalData)

print(len(candidate_links))

47217


In [ ]:
# Overall Algorithms needs more study and research to improve performance.

In [ ]:
# Compare records using the Jaro-Winkler similarity on the 'Name' column
compare = recordlinkage.Compare()

compare.string('First Name', 'first_name', method='jarowinkler',threshold=(jaccard_similarity_first+0.5),label="first_name")
compare.string('Last Name', 'last_name', method='jarowinkler',threshold=(jaccard_similarity_last+.5),label="last_name")

compare.exact('phonetic_given_name', 'phonetic_given_name', label="phonetic_given_name")
compare.exact('phonetic_last_name', 'phonetic_last_name', label='phonetic_surname')

compare.string('Address Line 1','addr 1', method='jarowinkler',threshold=0.65,label="address")
compare.string('City', 'city', method='jarowinkler',threshold=0.85,label="city")
compare.string('State','state', method='jarowinkler',threshold=0.85,label="state")

#compare.exact('phonetic_city', 'phonetic_city', label="phonetic_city")
#compare.exact('phonetic_state', 'phonetic_state', label='phonetic_state')

# Compute the potential matches
features = compare.compute(candidate_links, Train,originalData)
index = pd.MultiIndex.from_frame(TrainGT, names=['ID_1', 'ID_2'])

In [ ]:
print(features.sum(axis=1).value_counts().sort_index(ascending=False))
#Counting score for each

5.0        1
4.0       45
3.0      133
2.0     1391
1.0    45647
dtype: int64


In [ ]:
potential_matches = features[features.sum(axis=1) > 1]#removing optins with less score
#potential_matches['Score'] = potential_matches.loc[:, 'phonetic_given_name':'phonetic_surname'].sum(axis=1)
print(potential_matches)

                  first_name  last_name  phonetic_given_name  \
Cust ID    id                                                  
CustID_001 91632         0.0        0.0                    1   
CustID_002 51253         0.0        0.0                    1   
           52441         0.0        0.0                    1   
CustID_006 68911         0.0        0.0                    1   
CustID_007 91632         0.0        0.0                    1   
...                      ...        ...                  ...   
CustID_298 65526         0.0        0.0                    1   
           65541         0.0        0.0                    1   
           65570         0.0        0.0                    1   
           66065         0.0        0.0                    1   
           67789         0.0        0.0                    1   

                  phonetic_surname  address  city  state  
Cust ID    id                                             
CustID_001 91632                 1      0.0   0.0

In [ ]:
# Try all scores 0 to 7 in order to see the effect on precision and recall...
for i in range(5):
    matches = features[features.sum(axis=1) > i]
    pre = recordlinkage.precision(index, matches)
    rcl = recordlinkage.recall(index, matches)
    print("when score is " + str(i) + " precision is " + str(pre) + " and recall is " + str(rcl))

when score is 0 precision is 0.002583815151322617 and recall is 0.7625
when score is 1 precision is 0.05605095541401274 and recall is 0.55
when score is 2 precision is 0.329608938547486 and recall is 0.36875
when score is 3 precision is 0.6304347826086957 and recall is 0.18125
when score is 4 precision is 1.0 and recall is 0.00625


In [ ]:
# Select and print the matches
#matches = potential_matches[potential_matches.sum(axis=1) >= 2]
#print("Potential Matches:",matches)
classifier = recordlinkage.LogisticRegressionClassifier()
classifier.fit(potential_matches,match_index=index)

In [ ]:
# Predict the matches
predicted_matches = classifier.predict(potential_matches)


# Print the predicted matches
print("\nPredicted Matches:")
#print(predicted_matches)
print("precision: " + str(recordlinkage.precision(index, predicted_matches)))
print("recall: " + str(recordlinkage.recall(index, predicted_matches)))

#

F1score = 2 * (recordlinkage.precision(index, predicted_matches) * recordlinkage.recall(index, predicted_matches)) / (recordlinkage.precision(index, predicted_matches)+ recordlinkage.recall(index, predicted_matches))

print("F1-Score: " + str(F1score))



Predicted Matches:
precision: 0.782608695652174
recall: 0.1125
F1-Score: 0.19672131147540983


In [ ]:

import csv
with open('/content/linkageTrain.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(predicted_matches)

In [ ]:
clasifier2 = recordlinkage.SVMClassifier()
clasifier2.fit(potential_matches, index)
predictions = clasifier2.predict(potential_matches)

print("precision: " + str(recordlinkage.precision(index, predictions)))
print("recall: " + str(recordlinkage.recall(index, predictions)))

precision: 0.782608695652174
recall: 0.1125


In [ ]:

from recordlinkage.index import Block

# Add a phonetic version of the first and last names to the two datasets
Test["phonetic_given_name"] = phonetic(Test["First Name"], "soundex")
originalData["phonetic_given_name"] = phonetic(originalData["first_name"], "soundex")

Test["phonetic_last_name"] = phonetic(Test["Last Name"], "soundex")
originalData["phonetic_last_name"] = phonetic(originalData["last_name"], "soundex")

Test["phonetic_city"] = phonetic(Test["City"], "soundex")
originalData["phonetic_city"] = phonetic(originalData["city"], "soundex")

Test["phonetic_state"] = phonetic(Test["State"], "soundex")
originalData["phonetic_state"] = phonetic(originalData["state"], "soundex")

#Test["initials"] = (Test"First Name"].str[0] + Test["Last Name"].str[0])
#originalData["initials"] = (originalData["first_name"].str[0] + originalData["last_name"].str[0])


indexer = recordlinkage.Index()
#indexer.block("initials")
indexer.add(Block('phonetic_given_name', 'phonetic_given_name'))
indexer.add(Block('phonetic_last_name', 'phonetic_last_name'))
candidate_links = indexer.index(Test,originalData)


In [ ]:
features = compare.compute(candidate_links, Test,originalData)
indexTest = pd.MultiIndex.from_frame(TestGT, names=['ID_1', 'ID_2'])

In [ ]:
potential_matches = features[features.sum(axis=1) > 0]#removing optins with less score

In [ ]:
# Predict the matches
predicted_matches = classifier.predict(potential_matches)


# Print the predicted matches
print("\nPredicted Matches:")
#print(predicted_matches)
print("precision: " + str(recordlinkage.precision(index, predicted_matches)))
print("recall: " + str(recordlinkage.recall(index, predicted_matches)))

#

F1score = 2 * (recordlinkage.precision(index, predicted_matches) * recordlinkage.recall(index, predicted_matches)) / (recordlinkage.precision(index, predicted_matches)+ recordlinkage.recall(index, predicted_matches))

print("F1-Score: " + str(F1score))



Predicted Matches:
precision: 0.0
recall: 0.0


ZeroDivisionError: ignored

In [ ]:

predictions = clasifier2.predict(potential_matches)

print("precision: " + str(recordlinkage.precision(index, predictions)))
print("recall: " + str(recordlinkage.recall(index, predictions)))

NameError: ignored

In [ ]:
!pip3 install fuzzymatcher

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 13.9 MB/s eta 0:00:00
  Created wheel for metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13901 sha256=caebd0754f55ba977450741fc7071daaaa2a78e2f93cea72d7a525f2e0fe20e5
  Stored in directory: /root/.cache/pip/wheels/23/dd/1d/6cdd346605db62bde1f60954155e9ce48f4681c243f265b704
Successfully built metaphone


In [ ]:
import pandas as pd
from pathlib import Path
import fuzzymatcher
fuzzyTrain= TrainData
fuzzyOrig = originalDataT

In [ ]:
left_on = ['First Name', 'Last Name','Address Line 1','City', 'State', 'Zip', 'DOB']

right_on = [ 'first_name', 'last_name','addr 1', 'city', 'state', 'zip', 'DOB'
]

In [ ]:
matched_results = fuzzymatcher.fuzzy_left_join(fuzzyTrain,
                                            fuzzyOrig ,
                                            left_on,
                                            right_on,
                                                left_id_col='Cust ID',
                                            right_id_col='id'
                                            )



In [ ]:
result = matched_results[['Cust ID', 'id']]
matchedAns = pd.MultiIndex.from_frame(result, names=['ID_1', 'ID_2'])

In [ ]:
print(matchedAns)

MultiIndex([('CustID_101',  1824),
            ('CustID_102', 43207),
            ('CustID_103', 27213),
            ('CustID_104', 43210),
            ('CustID_106', 43208),
            ('CustID_107', 43210),
            ('CustID_108',    35),
            ('CustID_109',  4488),
            ('CustID_110', 51646),
            ('CustID_111', 39934),
            ...
            ('CustID_289', 66178),
            ('CustID_290', 51606),
            ('CustID_291',  8512),
            ('CustID_292', 38655),
            ('CustID_293', 52922),
            ('CustID_294',  8513),
            ('CustID_295', 52189),
            ('CustID_296', 21308),
            ('CustID_297', 48958),
            ('CustID_298', 65240)],
           names=['ID_1', 'ID_2'], length=160)


In [ ]:
print(TrainGT)

        Cust ID  Credit Bureau ID
0    CustID_101              1824
1    CustID_102             43207
2    CustID_103             27213
3    CustID_104             43208
4    CustID_106             43209
..          ...               ...
155  CustID_294              8513
156  CustID_295             52189
157  CustID_296             21308
158  CustID_297             48958
159  CustID_298             65240

[160 rows x 2 columns]


In [ ]:
print("precision: " + str(recordlinkage.precision(index, matchedAns)))
print("recall: " + str(recordlinkage.recall(index, matchedAns)))

precision: 0.825
recall: 0.825


In [ ]:
matched_results = fuzzymatcher.fuzzy_left_join(TestData,
                                            fuzzyOrig ,
                                            left_on,
                                            right_on,
                                                left_id_col='Cust ID',
                                            right_id_col='id'
                                            )


In [ ]:
result = matched_results[['Cust ID', 'id']]
matchedAns = pd.MultiIndex.from_frame(result, names=['ID_1', 'ID_2'])
indexTest = pd.MultiIndex.from_frame(TestGT, names=['ID_1', 'ID_2'])
print("precision: " + str(recordlinkage.precision(indexTest, matchedAns)))
print("recall: " + str(recordlinkage.recall(indexTest, matchedAns)))

precision: 0.9814814814814815
recall: 0.9814814814814815


In [ ]:
matchedAns
import csv
with open('/content/fuzzyTest.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(matchedAns)

In [ ]:
matched_results = fuzzymatcher.fuzzy_left_join(Val,
                                            fuzzyOrig ,
                                            left_on,
                                            right_on,
                                                left_id_col='Cust ID',
                                            right_id_col='id'
                                            )


In [ ]:
result = matched_results[['Cust ID', 'id']]
matchedAns = pd.MultiIndex.from_frame(result, names=['ID_1', 'ID_2'])

In [ ]:
with open('/content/fuzzyVal.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(matchedAns)